In [192]:
import numpy as np
import pandas as pd
import graphlab
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

BASE  = './bytecup2016data'
IINFO = BASE + '/invited_info_train.txt'
QINFO = BASE + '/question_info.txt'
VAL   = BASE + '/validate_nolabel.txt'
TEST   = BASE + '/test_nolabel.txt'

invdata = pd.read_csv(IINFO, delim_whitespace=True, header=None, names=["qid", "uid", "label"])
qdata   = pd.read_csv(QINFO, delim_whitespace=True, header=None, names=["qid", "qtag", "wseq", "cseq", "nvotes", "nans", "ntqans"])
valdata = pd.read_csv(TEST)

    
qdata = qdata.drop(["qtag","wseq","cseq"],axis = 1)

for col in ['nvotes', 'nans', 'ntqans']:
    qdata[col] = (qdata[col] - qdata[col].min())/(qdata[col].max() - qdata[col].min())
    
qdata = qdata.drop(["nans","ntqans"],axis = 1)

In [193]:
def algorithm_predict(new_invdata, new_valdata):
    sf = graphlab.SFrame(new_invdata, format='dataframe')
    side_item = graphlab.SFrame(qdata, format='dataframe')
    #m = graphlab.recommender.create(sf, target='rating')
    m = graphlab.item_similarity_recommender.create(sf, user_id='qid', item_id='uid', 
                                                                      target='label', similarity_type = 'pearson')
    #m = graphlab.factorization_recommender.create(sf, target='label', regularization = 5e-2 ,user_id='uid', item_id='qid')
    #m = graphlab.recommender.ranking_factorization_recommender.create(sf, user_id='uid', item_id='qid', 
                                                            #regularization = 1e-2 ,target='label')
    #m = graphlab.popularity_recommender.create(sf, target='rating')  
    
    sf = graphlab.SFrame(new_valdata, format='dataframe')
    return m.predict(sf)

In [194]:
np.random.seed(123)
skf = KFold(n_splits = 5)
invdata_label = invdata['label']
count = 1
for train_invdata, test_valdata in skf.split(invdata):
    new_invdata = invdata.loc[train_invdata]
    new_valdata = invdata.loc[test_valdata]
    new_invdata.reset_index(drop = True, inplace = True)
    new_valdata.reset_index(drop = True, inplace = True)
    
    new_valdata['label'] = algorithm_predict(new_invdata, new_valdata)
    
    new_valdata.to_csv('cross-validation-attempt' + str(count) + '.csv', separator=",")
    count += 1

Recsys training: model = item_similarity

Preparing data set.

Data has 196601 observations with 7572 users and 25693 items.

Data prepared in: 0.347104s

Training model from provided data.

...

Processing data in one pass using dense lookup tables.

+-------------------------------------+------------------+-----------------+

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

+-------------------------------------+------------------+-----------------+

MemoryError: std::bad_alloc